#### Configure Iceberg catalog

An Iceberg catalog contains the Iceberg table data and metadata. 
There are various implementations of the catalog, including REST and JDBC.

In this case, a `hadoop` catalog type is used to define a catalog called `local`.

A directory called `spark-warehouse/iceberg` is then created on the local filesystem.

Within this directory, Iceberg table metadata and data is stored in a subdirectory for each table.


In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.5.2') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

# Verify Spark session creation
spark.sql("SHOW DATABASES").show()

25/01/03 13:57:15 WARN Utils: Your hostname, Bjorns-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.104 instead (on interface en0)
25/01/03 13:57:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Users/bjorn/Documents/baolsen/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/bjorn/.ivy2/cache
The jars for the packages stored in: /Users/bjorn/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-108b5d40-923f-48e5-a9e2-2e0c62dd088d;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 in central
downloading https://repo1.maven.org/maven2/org/apache/iceberg/iceberg-spark-runtime-3.5_2.12/1.5.2/iceberg-spark-runtime-3.5_2.12-1.5.2.jar ...
	[SUCCESSFUL ] org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2!iceberg-spark-runtime-3.5_2.12.jar (4354ms)
:: resolution report :: resolve 1922ms :: artifacts dl 4363ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.5.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicte

+---------+
|namespace|
+---------+
|  default|
+---------+



#### Create and query an Iceberg table

In [2]:
# Create an Iceberg table
spark.sql("""
  CREATE TABLE local.schema.users (
    id INT,
    name STRING,
    age INT
  ) USING iceberg""")

# Insert some sample data
spark.sql("""
  INSERT INTO local.schema.users VALUES
    (1, 'Alice', 30),
    (2, 'Bob', 25),
    (3, 'Charlie', 35)""")

# Query the data
result = spark.sql("SELECT * FROM local.schema.users")
result.show()

+---+-------+---+
| id|   name|age|
+---+-------+---+
|  1|  Alice| 30|
|  2|    Bob| 25|
|  3|Charlie| 35|
+---+-------+---+

